# Distance to User Sort

This script provides two methods of finding the closest store based on gps coordinates of both the user and stores.

The first is a rough "as-the-crow-flies" method directly using GPS coordinates to calculate the absolute distance and returning the closest 3. This uses the Haversine formula.

The second uses OpenRouteService to find a route to the top result and provides an estimated time. This should be altered to test all 3 to ensure that the suggest closest store is the quickest to get to but the free tier has API limitations.

## Test Data

In [63]:
User_Location = [-37.84888732062262, 145.11313019713904]

Coles_1 = [-37.852773120693506, 145.14954100250657, "Coles Burwood East"]
Coles_2 = [-37.833718017777876, 145.07741030340782, "Coles Middle Camberwell"]
Coles_3 = [-37.81995969557401, 145.12026707669838, "Coles Box Hill Central"]
Woolworths_1 = [-37.85228698325045, 145.13439291289956, "Woolworths Burwood East"]
Woolworths_2 = [-37.820023903921175, 145.12097152835932, "Woolworths Box Hill"]
Woolworths_3 = [-37.84515592002607, 145.04237351854925, "Woolworths Hawthorn East"]
Route_Test_Store = [-37.6511209632499, 145.0743361878805, "Woolworths Plenty Valley"]

All_Stores = [Coles_1, Coles_2, Coles_3, Woolworths_1, Woolworths_2, Woolworths_3]

## GPS Method

In [64]:
from math import radians, cos, sin, asin, sqrt

# https://tinyurl.com/36umzjkx
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in km
    return c * r

In [65]:
# Testing

i = 0
while i < len(All_Stores):
    current_store = All_Stores[i]
    dist = haversine(User_Location[1],User_Location[0], current_store[1],current_store[0])
    dist = round(dist,3)
    print("The distance to " + current_store[2] + " is " + str(dist))
    i = i + 1

The distance to Coles Burwood East is 3.226
The distance to Coles Middle Camberwell is 3.561
The distance to Coles Box Hill Central is 3.277
The distance to Woolworths Burwood East is 1.905
The distance to Woolworths Box Hill is 3.283
The distance to Woolworths Hawthorn East is 6.227


# OpenRoute Method

In [66]:
import openrouteservice as ors
import folium

In [67]:
client = ors.Client(key = '5b3ce3597851110001cf624874854789b90143ffafc81567715dbc6e')

In [68]:
# Folium goes lat, long while ors does long,lat so you have to reverse coordinates between the two.
# The test data is in lat,long 

m = folium.Map(location = list((User_Location)), tiles = "cartodbpositron", zoom_start = 11)

route_ends = [User_Location[0:2][::-1],Route_Test_Store[0:2][::-1]]

# Using car profile (Public transit is not offered by ORS)
route = client.directions(coordinates = route_ends, profile = 'driving-car', format = 'geojson')

folium.PolyLine(locations = [list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']], color = 'blue').add_to(m)

m

As the route is in the GeoJSON format, we can extract the route for directions or export the waypoints for the route without requerying. The two biggest problems with the above method is the lack of support for public transit and no traffic indicators. There are alternatives that support this but they are paid so I couldn't test them.